In [ ]:
from loguru import logger
from api_models import get_models, ModelListResponse,ModelPermission,Model
response: ModelListResponse = await get_models()
model: Model = response.data[0]

%load_ext autoreload
%autoreload 2

In [ ]:
model

In [ ]:
from api_utils import wrap_and_print,  get_default_sample_params, generate
import json 
prompt = ["San Francisco is a"]  # Note: prompt is a list
params: SamplingParams = get_default_sample_params()
logger.info(params)
async for completion in generate(params, "ehartford/dolphin-2.1-mistral-7b", prompt):
    print(completion)

In [66]:
import openai 
openai.api_key = "EMPTY"
openai.api_base = "http://127.0.0.1:8100/v1"
import asyncio
from openai_streaming import process_response
from typing import AsyncGenerator

# Define content handler
async def content_handler(content: AsyncGenerator[str, None]):
    async for token in content:
        print(token, end="")

print(f"USING MODEL : {model['id']}")
description = "Create a Python script to sort a list of numbers in ascending order."
response = openai.Completion.create(
  model=model["id"],
  prompt=f"Instruction:\n{description}",
  
  max_tokens=2000
)

code = response.choices[0].text.strip()
print(code)




USING MODEL : TheBloke/Phind-CodeLlama-34B-v2-AWQ
Sure, here is a simple Python script that uses the built-in `sorted()` function to sort a list of numbers in ascending order:

```python
# Here's your list of numbers
numbers = [56, 19, 33, 71, 28, 45, 87, 43, 54, 22]

# This function will sort the list
sorted_numbers = sorted(numbers)

# Now print out the sorted list
print("Sorted numbers list is:", sorted_numbers)
```

If you run this script, you will see that it prints out the sorted list:

```python
Sorted numbers list is: [19, 22, 28, 33, 43, 45, 54, 56, 71, 87]
```

If you wish to sort the list in place (meaning the original list is sorted, not creating a new list), you can use the `list.sort()` method:

```python
numbers = [56, 19, 33, 71, 28, 45, 87, 43, 54, 22]
numbers.sort()
print("Sorted numbers list is:", numbers)
```

This would also print:

```python
Sorted numbers list is: [19, 22, 28, 33, 43, 45, 54, 56, 71, 87]
```


In [ ]:
def sort_numbers(numbers):
    return sorted(numbers)

# Use example
numbers = [67, 23, 34, 2, 89, 23, 2]
sorted_numbers = sort_numbers(numbers)
print(sorted_numbers)

In [ ]:
description

In [64]:
# Example of an OpenAI Completion request, using the stream=True option
# https://beta.openai.com/docs/api-reference/completions/create
import time
# record the time before the request is sent
start_time = time.time()

# send a Completion request to count to 100
response = openai.Completion.create(
    model=model['id'],
    prompt='1,2,3,',
    max_tokens=2000,
    temperature=0,
    stream=True,  # this time, we set stream=True
)

# Iterate through the stream of events
for event in response:
    event_text = event['choices'][0]['text']  # Extract the text from the event
    collected_events.append(event)  # Collect the event (optional, if you need it later)
    
    # Append the streamed text to the existing output
    completion_text += event_text
    
    # Print the updated completion_text
    print(completion_text)


4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,14,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,334
4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,14,5,6,7,8,9,10,11,12,

KeyboardInterrupt: 

In [63]:
import gradio as gr
import os 
import json 
import requests

#Streaming endpoint 
API_URL = "http://127.0.0.1:8100/v1/chat/completions" #os.getenv("API_URL") + "/generate_stream"

#Inferenec function
def predict(system_msg, inputs, top_p, temperature, chat_counter, chatbot=[], history=[]):  

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_gpt4_key}"  #Users will provide their own OPENAI_API_KEY 
    }
    print(f"system message is ^^ {system_msg}")
    if system_msg.strip() == '':
        initial_message = [{"role": "user", "content": f"{inputs}"},]
        multi_turn_message = []
    else:
        initial_message= [{"role": "system", "content": system_msg},
                   {"role": "user", "content": f"{inputs}"},]
        multi_turn_message = [{"role": "system", "content": system_msg},]
        
    if chat_counter == 0 :
        payload = {
        "model": model['id'],
        "messages": initial_message , 
        "temperature" : 1.0,
        "top_p":1.0,
        "n" : 1,
        "stream": True,
        "presence_penalty":0,
        "frequency_penalty":0,
        }
        print(f"chat_counter - {chat_counter}")
    else: #if chat_counter != 0 :
        messages=multi_turn_message # Of the type of - [{"role": "system", "content": system_msg},]
        for data in chatbot:
          user = {}
          user["role"] = "user" 
          user["content"] = data[0] 
          assistant = {}
          assistant["role"] = "assistant" 
          assistant["content"] = data[1]
          messages.append(user)
          messages.append(assistant)
        temp = {}
        temp["role"] = "user" 
        temp["content"] = inputs
        messages.append(temp)
        #messages
        payload = {
        "model": model['id'],
        "messages": messages, # Of the type of [{"role": "user", "content": f"{inputs}"}],
        "temperature" : temperature, #1.0,
        "top_p": top_p, #1.0,
        "n" : 1,
        "stream": True,
        "presence_penalty":0,
        "frequency_penalty":0,}

    chat_counter+=1

    history.append(inputs)
    print(f"Logging : payload is - {payload}")
    # make a POST request to the API endpoint using the requests.post method, passing in stream=True
    response = requests.post(API_URL, headers=headers, json=payload, stream=True)
    print(f"Logging : response code - {response}")
    token_counter = 0 
    partial_words = "" 

    counter=0
    for chunk in response.iter_lines():
        #Skipping first chunk
        if counter == 0:
          counter+=1
          continue
        # check whether each line is non-empty
        if chunk.decode() :
          chunk = chunk.decode()
          # decode each line as response data is in bytes
          if len(chunk) > 12 and "content" in json.loads(chunk[6:])['choices'][0]['delta']:
              partial_words = partial_words + json.loads(chunk[6:])['choices'][0]["delta"]["content"]
              if token_counter == 0:
                history.append(" " + partial_words)
              else:
                history[-1] = partial_words
              chat = [(history[i], history[i + 1]) for i in range(0, len(history) - 1, 2) ]  # convert to tuples of list
              token_counter+=1
              yield chat, history, chat_counter, response  # resembles {chatbot: chat, state: history}  
                   
#Resetting to blank
def reset_textbox():
    return gr.update(value='')

#to set a component as visible=False
def set_visible_false():
    return gr.update(visible=False)

#to set a component as visible=True
def set_visible_true():
    return gr.update(visible=True)

#Using info to add additional information about System message in GPT4
system_msg_info = """You are an AI programming assistant.
        
                - Follow the user's requirements carefully and to the letter.
                - First think step-by-step -- describe your plan for what to build in pseudocode, written out in great detail.
                - Then output the code in a single code block.
                - Minimize any other prose."""
      

with gr.Blocks(css = """#col_container { margin-left: auto; margin-right: auto;} #chatbot {height: 1520px; overflow: auto;}""",) as demo:
    with gr.Column(elem_id = "col_container"):
        #Users need to provide their own GPT4 API key, it is no longer provided by Huggingface 
        with gr.Row():
            openai_gpt4_key = ""
            with gr.Accordion(label="System message:", open=False):
                system_msg = gr.Textbox(label="Instruct the AI Assistant to set its beaviour", info = system_msg_info, value="",placeholder="Type here..")
                accordion_msg = gr.HTML(value="🚧 To set System message you will have to refresh the app", visible=False)
                          
        chatbot = gr.Chatbot(label='GPT4', elem_id="chatbot")
        inputs = gr.Textbox(placeholder= "Hi there!", label= "Type an input and press Enter")
        state = gr.State([]) 
        with gr.Row():
            with gr.Column(scale=7):
                b1 = gr.Button().style(full_width=True)
            with gr.Column(scale=3):
                server_status_code = gr.Textbox(label="Status code from OpenAI server", )
    
        #top_p, temperature
        with gr.Accordion("Parameters", open=False):
            top_p = gr.Slider( minimum=-0, maximum=1.0, value=1.0, step=0.05, interactive=True, label="Top-p (nucleus sampling)",)
            temperature = gr.Slider( minimum=-0, maximum=5.0, value=1.0, step=0.1, interactive=True, label="Temperature",)
            chat_counter = gr.Number(value=0, visible=False, precision=0)

    #Event handling
    inputs.submit( predict, [system_msg, inputs, top_p, temperature, chat_counter, chatbot, state], [chatbot, state, chat_counter, server_status_code],)  #openai_api_key
    b1.click( predict, [system_msg, inputs, top_p, temperature, chat_counter, chatbot, state], [chatbot, state, chat_counter, server_status_code],)  #openai_api_key
    
    inputs.submit(set_visible_false, [], [system_msg])
    b1.click(set_visible_false, [], [system_msg])
    inputs.submit(set_visible_true, [], [accordion_msg])
    b1.click(set_visible_true, [], [accordion_msg])
    
    b1.click(reset_textbox, [], [inputs])
    inputs.submit(reset_textbox, [], [inputs])

demo.queue(max_size=99, concurrency_count=20).launch(debug=True)
%%javascript
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 444em; }</style>"))

/home/jphillips/fearnworks/aidriver/.venv/lib/python3.11/site-packages/gradio/components/button.py:112: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(
/home/jphillips/fearnworks/aidriver/.venv/lib/python3.11/site-packages/gradio/components/button.py:116: UserWarning: Use `scale` in place of full_width in the constructor. scale=1 will make the button expand, whereas 0 will not.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


system message is ^^ 
chat_counter - 0
Logging : payload is - {'model': 'TheBloke/Phind-CodeLlama-34B-v2-AWQ', 'messages': [{'role': 'user', 'content': 'How does RabbitMQ work?'}], 'temperature': 1.0, 'top_p': 1.0, 'n': 1, 'stream': True, 'presence_penalty': 0, 'frequency_penalty': 0}
Logging : response code - <Response [200]>
Keyboard interruption in main thread... closing server.


UsageError: Line magic function `%%javascript` not found.


In [44]:


demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
